In [2]:
from PIL import Image, ImageDraw, ImageFont
import PIL.ImageOps
#import cv2
import numpy as np
import time
from threading import Thread
from multiprocessing import Pool, freeze_support
from functools import partial
import math
import csv



In [3]:
def in_csv(letters):
    FILENAME = 'letters2.csv'
    with open(FILENAME, "w", newline="") as file:
        writer = csv.writer(file,delimiter=';')
        writer.writerows(letters)

In [27]:
def create_img():
    alphabeth = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
    font = ImageFont.truetype("ofont.ru_Times New Roman.ttf", 52)
    for letter in alphabeth:
        img = Image.new("L", (60, 60), (0))
        draw = ImageDraw.Draw(img)
        w,h=font.getsize(letter)
        shift=12
        if letter == "I" or letter == "J":
            shift=20
        if letter == "W":
            shift=5
        
        draw.text((0+shift, 0), letter, (255), font=font)
        img.save('pictures/alphabeth/'+letter+'.bmp')

In [28]:
create_img()

In [6]:
def timeit(f):
    def wrap(*args):
        time_start = time.time()
        ret = f(*args)
        time_end = time.time()
        print((time_end-time_start)*1000.0)
        return ret
    return wrap

In [7]:
#надо инвертировать и f(x,y) от 0 до 1 1-черный, 0-белый как в лекциях
def to_inverted_grayscale(image):
    new_img = image.convert('L')
    new_img = PIL.ImageOps.invert(new_img)
    arr=np.asarray(new_img)
    return new_img

def get_black_weight(img):
    pix = np.asarray(img)
    return np.sum(pix)/255

def get_normal_black_weight(img):
    s = img.size[0] * img.size[1]
    return get_black_weight(img)/s

def get_moment(p,q,img, x_av=0, y_av=0):
    width=img.size[0]
    height = img.size[1]
    moment=0
    pix = np.asarray(img)
    for x in range(width):
        for y in range(height):
            moment += math.pow(x-x_av,p)*math.pow(y-y_av,q)*pix[x][y]
    return moment/255

def get_weight_center(img):
    black_w = get_black_weight(img)
    x_coords = get_moment(1,0,img)/black_w
    y_coords = get_moment(0,1,img)/black_w
    return (x_coords,y_coords)
#нормированные координаты центра тяжести(от 0 до 1)
def get_normal_weight_center(img,x_coords,y_coords):
    width = img.size[0]
    height = img.size[1]
    x_coords_norm = (x_coords-1)/(width-1)
    y_coords_norm = (y_coords - 1) / (height - 1)
    return (x_coords_norm,y_coords_norm)

In [8]:
#осевые моменты инерции по горизонтали и вертикали
def get_inert_axis(img,x_center,y_center):
    x_axis = get_moment(0,2,img,x_av=0,y_av=y_center)
    y_axis = get_moment(2,0,img,x_av=x_center,y_av=0)
    return (x_axis,y_axis)
#нормированные осевые моменты инерции
def get_normal_inert_axis(x_axis,y_axis,img):
    width = img.size[0]
    height = img.size[1]
    norm=math.pow(width,2)+math.pow(height,2)
    return (x_axis/norm,y_axis/norm)

In [9]:
@timeit
def main():
    alphabeth = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
    d=[]
    black_weight_list=["black_weight_list"]
    normal_black_weight_list = ["normal_black_weight_list"]

    weight_center_x_axis = ["weight_center_x"]
    weight_center_y_axis = ["weight_center_y"]

    normal_weight_center_x_axis = ["normal_weight_center_x"]
    normal_weight_center_y_axis = ["normal_weight_center_y"]

    inert_axis_x = ["inert_axis_x"]
    inert_axis_y = ["inert_axis_y"]

    normal_inert_axis_x = ["normal_inert_axis_x"]
    normal_inert_axis_y = ["normal_inert_axis_y"]
    f='.3f'
    for letter in alphabeth:

        path_to_img = "pictures/alphabeth/"+letter+".bmp"
        image = Image.open(path_to_img).convert("L")
        black_weight_list.append(format(get_black_weight(image),f))
        normal_black_weight_list.append(format(get_normal_black_weight(image),f))

        (x,y) = get_weight_center(image)
        weight_center_x_axis.append(format(x,f))
        weight_center_y_axis.append(format(y,f))

        (x_norm,y_norm) =get_normal_weight_center(image, x, y)
        normal_weight_center_x_axis.append(format(x_norm,f))
        normal_weight_center_y_axis.append(format(y_norm,f))

        (x_ax,y_ax) = get_inert_axis(image, x, y)
        inert_axis_x.append(format(x_ax,f))
        inert_axis_y.append(format(y_ax,f))

        (x_norm_ax, y_norm_ax) = get_normal_inert_axis(x_ax,y_ax,image)
        normal_inert_axis_x.append(format(x_norm_ax,f))
        normal_inert_axis_y.append(format(y_norm_ax,f))

    d.append(black_weight_list)
    d.append(normal_black_weight_list)
    d.append(weight_center_x_axis)
    d.append(weight_center_y_axis)
    d.append(normal_weight_center_x_axis)
    d.append(normal_weight_center_y_axis)
    d.append(inert_axis_x)
    d.append(inert_axis_y)
    d.append(normal_inert_axis_x)
    d.append(normal_inert_axis_y)
    in_csv(d)


In [10]:

def main2():
    alphabeth = list("АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ")
    d=[]
    f='.3f'

    head = ["l/p","black_weight","normal_black_weight","weight_center_x","weight_center_y","normal_weight_center_x","normal_weight_center_y",
            "inert_axis_x","inert_axis_y","normal_inert_axis_x","normal_inert_axis_y"]
    d.append(head)
    for letter in alphabeth:
        path_to_img = "pictures/alphabeth/"+letter+".bmp"
        image = Image.open(path_to_img).convert("L")

        row = [letter]

        row.append(format(get_black_weight(image),f))
        row.append(format(get_normal_black_weight(image),f))

        (x,y) = get_weight_center(image)
        row.append(format(x,f))
        row.append(format(y,f))

        (x_norm,y_norm) =get_normal_weight_center(image, x, y)
        row.append(format(x_norm,f))
        row.append(format(y_norm,f))

        (x_ax,y_ax) = get_inert_axis(image, x, y)
        row.append(format(x_ax,f))
        row.append(format(y_ax,f))

        (x_norm_ax, y_norm_ax) = get_normal_inert_axis(x_ax,y_ax,image)
        row.append(format(x_norm_ax,f))
        row.append(format(y_norm_ax,f))

        d.append(row)
    in_csv(d)



In [15]:
#
#main2()
create_img()